In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.datasets import fetch_openml


In [2]:
# Cargar el conjunto de datos MNIST
mnist = fetch_openml('mnist_784', version=1)
X, y = mnist["data"], mnist["target"]

# Normalizar las imágenes de 0 a 1
X = X.astype('float32') / 255.0

# Convertir las etiquetas a enteros y luego a categóricas (one-hot encoding)
y = to_categorical(y.astype('int'))



h:\Anaconda\envs\IA\lib\site-packages\sklearn\datasets\_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [3]:
def create_model(optimizer='adam', init='glorot_uniform', activation='relu'):
    model = Sequential()
    model.add(Flatten(input_shape=(784,)))
    model.add(Dense(128, kernel_initializer=init, activation=activation))
    model.add(Dense(64, kernel_initializer=init, activation=activation))
    model.add(Dense(10, activation='softmax'))
    
    # Compile the model
    model.compile(optimizer=optimizer, 
                  loss='categorical_crossentropy', 
                  metrics=['accuracy'])
    
    return model



In [4]:
# Define the grid of hyperparameters
param_grid = {
    'batch_size': [32, 64, 128],
    'epochs': [10, 20],
    'optimizer': ['adam'],
    'init': ['normal', 'glorot_uniform'],
    'activation': ['relu', 'tanh']
}

# Implement K-Fold Cross-Validation
kf = KFold(n_splits=3)

best_params = None
best_score = 0

for train_index, val_index in kf.split(X_train):
    X_train_k, X_val_k = X_train[train_index], X_train[val_index]
    y_train_k, y_val_k = y_train[train_index], y_train[val_index]
    
    params, score = manual_grid_search(X_train_k, y_train_k, X_val_k, y_val_k, param_grid)
    
    if score > best_score:
        best_score = score
        best_params = params

print(f"\nBest Score: {best_score:.4f} with Best Params: {best_params}")

C:\Users\yoda\AppData\Local\Temp\ipykernel_24980\2936750297.py:2: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=create_model, verbose=0)


In [5]:
# Train the final model using the best hyperparameters
final_model = create_model(optimizer=best_params['optimizer'], 
                           init=best_params['init'], 
                           activation=best_params['activation'])

final_model.fit(X_train, y_train, 
                epochs=best_params['epochs'], 
                batch_size=best_params['batch_size'], 
                verbose=1)

# Evaluate the final model on the test set
test_score = final_model.evaluate(X_test, y_test, verbose=0)
print(f"\nFinal Test Accuracy: {test_score[1]:.4f}")


BrokenProcessPool: A task has failed to un-serialize. Please ensure that the arguments of the function are all picklable.